<a href="https://colab.research.google.com/github/kevingray92/597_Project4_GroupB/blob/checkwithnewCMI_hourly_and_GFS_sfc/KCMI_parsing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import os
# import tarfile

### Adding hourly KCMI precip to the daily data

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
#Download the data
!wget https://raw.githubusercontent.com/swnesbitt/ATMS-597-SP-2020/master/ATMS-597-SP-2020-Project-4/KCMI_hourly.csv

--2020-03-30 18:44:48--  https://raw.githubusercontent.com/swnesbitt/ATMS-597-SP-2020/master/ATMS-597-SP-2020-Project-4/KCMI_hourly.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7611908 (7.3M) [text/plain]
Saving to: ‘KCMI_hourly.csv’

KCMI_hourly.csv     100%[===================>]   7.26M  --.-KB/s    in 0.1s    

2020-03-30 18:44:49 (58.6 MB/s) - ‘KCMI_hourly.csv’ saved [7611908/7611908]



In [ ]:
#Used to unzip the .tar.gz files
!gunzip drive/My\ Drive/Colab\ Notebooks/ATMS-597/Module04/Project04/sfc.tar.gz

In [ ]:
#Use this to untar the .tar files
#Looks like it will send all of these to a new remote directory called 'bufkit'
#Might want to specify a destination folder in the future
!tar -xvf drive/My\ Drive/Colab\ Notebooks/ATMS-597/Module04/Project04/sfc.tar

In [8]:
#  dailyobs = pd.read_csv('drive/My Drive/KCMI_dailyNOFOOTER.csv', header=8)  # Read the daily obs (removed the footer in the original file)
dailyobs = pd.read_csv('./raw_data/KCMI_dailyNOFOOTER.csv', header=8)  # (David) Read the daily obs (removed the footer in the original file)
dailyobs.drop(['Total Precip (in)'],axis=1)  # Drop the daily precip column that we are going to replace
dailyobs

,Date,Max Hourly Temp (F),Min Hourly Temp (F),Max Wind Speed (mph),Total Precip (in)
0,1/1/2010,12,2,15,M
1,1/2/2010,6,-5,15,0
2,1/3/2010,10,-6,11,0
3,1/4/2010,11,-2,20,0
4,1/5/2010,10,-1,16,M
...,...,...,...,...,...
3647,12/27/2019,43,32,15,0
3648,12/28/2019,56,32,23,0.23
3649,12/29/2019,59,41,23,0.22
3650,12/30/2019,38,31,26,0.01


In [10]:
# hourlyobs = pd.read_csv('drive/My Drive/KCMI_hourly.csv')  # Read the hourly file
hourlyobs = pd.read_csv('./raw_data/KCMI_hourly.csv')  # (David) Read the hourly file
hourlyprecip = hourlyobs.drop(['tmpc','dwpc','mslp','wdir','wspd','skct','pr6h','doy','woy'],axis=1)  # Only keep precip column
hourlyprecip.set_index('Timestamp')  # Make the index be the timestamp
hourlyprecip.index = pd.to_datetime(hourlyprecip['Timestamp'])  # Make the index be a datetime
dailyprecip = hourlyprecip.resample('d').sum()  # Resample to daily
dailyprecip.columns = ['Daily Precip (mm)']  # Rename the column

In [11]:
dailyprecip.head()

,Daily Precip (mm)
Timestamp,
2006-01-01,0.0
2006-01-02,2.3
2006-01-03,-0.1
2006-01-04,0.3
2006-01-05,0.0


In [12]:
dailyprecipshort = dailyprecip[(dailyprecip.index>'2009-12-31') & (dailyprecip.index<'2020-01-01')]  # Only get 2010 to 2019
dailyobs['Daily Precip (mm)'] = dailyprecipshort['Daily Precip (mm)'].values  # Put the new calculated daily precip in the dailyobs dataframe
dailyobs  # This should be the final dataframe used as the predictands

,Date,Max Hourly Temp (F),Min Hourly Temp (F),Max Wind Speed (mph),Total Precip (in),Daily Precip (mm)
0,1/1/2010,12,2,15,M,0.0
1,1/2/2010,6,-5,15,0,0.0
2,1/3/2010,10,-6,11,0,0.0
3,1/4/2010,11,-2,20,0,0.0
4,1/5/2010,10,-1,16,M,-0.1
...,...,...,...,...,...,...
3647,12/27/2019,43,32,15,0,0.0
3648,12/28/2019,56,32,23,0.23,0.0
3649,12/29/2019,59,41,23,0.22,6.1
3650,12/30/2019,38,31,26,0.01,1.0


In [13]:
dailyobs = dailyobs.drop(columns = ['Total Precip (in)']).replace('M', np.nan)

In [14]:
dailyobs = dailyobs.interpolate()

In [15]:
# Convert to deg C, m/s, and set negative precip values to zero
dailyobs['Max Hourly Temp (F)'] = (dailyobs['Max Hourly Temp (F)'].astype(float) - 32.) * 5./9.
dailyobs['Min Hourly Temp (F)'] = (dailyobs['Min Hourly Temp (F)'].astype(float) - 32.) * 5./9.
dailyobs['Max Wind Speed (mph)'] = dailyobs['Max Wind Speed (mph)'].astype(float) * 0.44704
dailyobs['Daily Precip (mm)'] = dailyobs['Daily Precip (mm)'].astype(float).apply(lambda x: 0.0 if x < 0.0 else x)

In [16]:
# Rename columns
dailyobs.rename(columns = {'Max Hourly Temp (F)' : 'Max Hourly Temp (C)', 'Min Hourly Temp (F)' : 'Min Hourly Temp (C)', 'Max Wind Speed (mph)' : 'Max Wind Speed (m/s)'}, inplace = True)

In [17]:
dailyobs

,Date,Max Hourly Temp (C),Min Hourly Temp (C),Max Wind Speed (m/s),Daily Precip (mm)
0,1/1/2010,-11.111111,-16.666667,6.70560,0.0
1,1/2/2010,-14.444444,-20.555556,6.70560,0.0
2,1/3/2010,-12.222222,-21.111111,4.91744,0.0
3,1/4/2010,-11.666667,-18.888889,8.94080,0.0
4,1/5/2010,-12.222222,-18.333333,7.15264,0.0
...,...,...,...,...,...
3647,12/27/2019,6.111111,0.000000,6.70560,0.0
3648,12/28/2019,13.333333,0.000000,10.28192,0.0
3649,12/29/2019,15.000000,5.000000,10.28192,6.1
3650,12/30/2019,3.333333,-0.555556,11.62304,1.0


In [18]:
# Save to csv
dailyobs.to_csv('./raw_data/KCMI_daily_tidy.csv', index = False)
# dailyobs.to_csv('drive/My Drive/KCMI_daily_tidy.csv', index = False)

### Parsing the hourly data

In [19]:
# Hourly data
#  hourlyobs = pd.read_csv('drive/My Drive/KCMI_hourly.csv')  # Read the hourly file
hourlyobs = pd.read_csv('./raw_data/KCMI_hourly.csv')  # (David) Read the hourly file

In [20]:
hourlyobs['Timestamp'] = pd.to_datetime(hourlyobs['Timestamp'])
hourlyobs.set_index('Timestamp', inplace = True)

In [21]:
hourlyobs = hourlyobs[(hourlyobs.index.hour >= 12) & (hourlyobs.index.hour <= 23)]

In [22]:
hourlyobs = hourlyobs.resample('D').mean()

In [23]:
hourlyobs.index = hourlyobs.index + pd.DateOffset(days = 1)

In [24]:
hourlyobs = hourlyobs[(hourlyobs.index > '2009-12-31') & (hourlyobs.index < '2020-01-01')]

In [25]:
hourlyobs = hourlyobs.drop(columns = ['pr6h', 'doy', 'woy'])

In [27]:
# hourlyobs.to_csv('drive/My Drive/KCMI_hourly_tidy.csv')
hourlyobs.to_csv('./raw_data/KCMI_hourly_tidy.csv')

In [26]:
hourlyobs

,tmpc,dwpc,mslp,wdir,wspd,skct,pr1h
Timestamp,,,,,,,
2010-01-01,-1.975000,-4.183333,1018.208333,279.166667,6.183333,6.000000,0.000000
2010-01-02,-13.325000,-16.933333,1029.316667,286.666667,5.591667,2.250000,0.000000
2010-01-03,-16.525000,-19.816667,1032.991667,296.666667,4.633333,0.000000,0.000000
2010-01-04,-15.325000,-19.858333,1032.250000,305.833333,3.641667,0.000000,0.000000
2010-01-05,-14.441667,-17.700000,1028.141667,290.833333,6.300000,1.333333,0.000000
...,...,...,...,...,...,...,...
2019-12-27,13.466667,11.141667,1016.258333,200.833333,6.125000,1.090909,0.000000
2019-12-28,2.816667,0.725000,1025.341667,242.500000,2.775000,0.666667,0.000000
2019-12-29,7.141667,6.866667,1018.491667,105.833333,5.975000,9.000000,0.000000
